bi= int(b) (big edian, les octets les plus à gauche sont les bits de points fort)
Pk=N/e
Sk=d/de

Signature de m:
h=sha256(m)
Construction de B points Bi
s=Bi mod N

Verification de s:
Ti=s**e mod N
Construction de T de même type que B

Faille :
e=3 et bug
Nombre d'octet max dans B = 256
Bug : T accepté également si il est du type :
B1=[000...1 *20 |FF...F sur 8 octet| 00...0*10|hash id * 190 | h * 320|junk * 1940] somme = 2048 le protocole compare les 62 premiers octets

Etapes : 
1) construire B1 donc B1i

2) calcul de s1 telle que s1/s1**3 = B1i sur les 62 premiers octets 

//la verif regarde que les 62 premiers octets 
verif buger(s1)
s1**3 == B1i sur les 62 premiers octets

Idée :
trouver s1 comme une racine cubique  entière de B1 mod N de moins de 682 bits = 2048/3
N contient 2048 bits donc |s1| <= 682 =>|s1**3| <= 2048
On se fou du modulo car N=2048 donc pour un entier assez petit ça sert à rien.

Soucis : si B1i=100 => 100**1/3 = 4,.... pas entier si on veut s=4 => s^3=64 < 100 toujours qqch de plus petit


Si s1**3 est plus grand que B1i entier tel que :
B1 entier =[000...1 *20 |FF...F sur 8 octet| 00...0*10|hash id * 190 | h * 320|junk * 1940]
alors gagné 
on veut que s1**3=B1i >= B1i entier 

Solution:
On prend B1i le plus grand possible c'est à, dire prendre le junk le plus grand possible (FFFFF...FFF)


In [ ]:
import math
from pkcs1 import *


def find_invpow(x,n):#on calcul s comme la racine cubique de B1 mod n
    """Finds the integer component of the n'th root of x,
    an integer such that y ** n <= x < (y + 1) ** n.
    """
    high = 1
    while high ** n < x:
        high *= 2
    low = high//2
    while low < high:
        mid = (low + high) // 2
        if low < mid and mid**n < x:
            low = mid
        elif high > mid and mid**n > x:
            high = mid
        else:
            return mid
    return mid + 1

M="PPTI SERVER ACCESS ON 2025-10-20 11:21 UTC"
n=0x00ed6b8f06d5b030d247e97994dd1d04f3016981dbde88e5b8f799a30817ce8f7097c6e8a0ba5f7579eba746e77d1bd3627d01cab793d1185710a9c4d8822310471bb60126f95bfc0dac38ee26c4fb6742d39ac49f0e828e4cdaf42f491b10cc8edd8474af88e6ed31a5f7de20b784260a8a70af6e0206606491a32b2ba3d6cc5065c31835a90bb9f33779622c8400b4a46c4a247d6aebf02235861353cb9ced948027706633944362bf6f823730dc7d9f6e5c2b52a88359fcfd182dbd1d753009030a6488aa160d021c715e35065d542c0f6f62ef5fd1fd02d4f80ecef6af2abf695cbfdfae2fe769034ef2849c9a92d010970a3739c0f37efb71afdf16abda4d
e=3
k = key_length(n)
by = emsa_pkcs1_encode(M,k)
x= os2ip(by)
S=i2osp(find_invpow(x,3),k)
print(emsa_pkcs1_decode(i2osp(pow(os2ip(S),3),k), k).hex())
print(sha256(M.encode()).digest().hex())

print(len(S))
print(i2osp(os2ip(S)**3,k).hex())
print(k)
print(S.hex())
#print(rsa_pkcs_verify(n,e,M,S))

c6de52c4355e15ac43abb5ab3487a2e7b7210aeb6d601dc15a0426efc0acfdc4
c6de52c4355e15ac43abb5ab3487a2e7b7210aeb6d601dc15a0426efc0acfdc4
256
0001ffffffffffffffffffff003031300d060960864801650304020105000420c6de52c4355e15ac43abb5ab3487a2e7b7210aeb6d601dc15a0426efc0acfdc4ffffffffffffffffffffffffffffffffffffffffffffeab8f2be36eda0185881649ef6a6fdd0efc19b719016851b4516a1699b891cf68f062b80295c00562434b37b5a54078b3cc93285d05b45394a88f51a7306cccfbd080057dcace7ee433225604d4a17b75e2a57492c22254e7eaa640800fafcca09cecb9c46dd6782905da8c783cdb93f312279481755b1e647618827f4ddcf676ae69accedf229f463f249117b19c266307b86357e1150271c00faf8c801c9aa4193bffeab36d50b0721
256
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000032cbfd4